In [1]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# some constants
beta = 4.
K0 = 1

upper_K = np.sqrt(2)
lower_K = 0.

def omega(K):
    return 1. - K^2 + 1j*np.sqrt(2 - *K^2)

def integrand(K):
    return np.exp(-beta^2*(K-K0)^2)*np.exp(-1j*K^2*tau/2.)*(np.exp(1j*K*u)-np.exp(-1j*K*u)*omega(K))

def integrand_square(u_input, tao_input):
    return np.conjugate(Psi(u_input, tao_input))*Psi(u_input, tao_input)

def Psi(u_input, tao_input):
    u, tao = u_input, tao_input
    return quad(integrand, lower_K, upper_K)[0]

In [7]:
tau_line = np.arange(-10, 10, 0.01)
u_line = np.arange(-10, 10, 0.01)